# AWS2input
Author: Anna Baldo - Master thesis - University of Innsbruck & IGF - anna.baldo@student.uibk.ac.at <br>

In [ ]:
# Import packages
import numpy as np
from matplotlib import pyplot as plt
import xarray as xr
import pandas as pd
import datetime

In [ ]:
# input file
filepath = 'your own filepath/20171031-20231031_WSS_Corrected.dat'
ds = pd.read_csv(filepath, sep = ',', skiprows = [0], parse_dates=True, encoding='latin-1')
ds.set_index('TIMESTAMP', inplace=True)
ds.index = pd.to_datetime(ds.index, format='%Y-%m-%d %H:%M:%S')

# clean LWout_corr columns
ds['LWout_corr'] = ds['LWout_corr'].where(ds['LWout_overmax_flag'] == 0).dropna()
# select only columns for cosipy
cols = ['Press_Avg', 'Tair', 'Hum', 'Wspeed', 'Wdir', # 'EisT1', 'EisT2', 'EisT3', 'EisT4',
       'SWout_albedo', 'SWin_albedo', 'LWin_corr', 'LWout_corr', 'snow_clean']
ds_cosipy = ds[cols]

In [ ]:
# Get start and end date for cutting precipitation variables
start_date = ds_cosipy.index[0].values
end_date = ds_cosipy.index[-1].values

## Albedo

In [33]:
# Add albaedo variable - used specifically in modified COSIPY version used in the Master thesis

# daily
albedo_daily = ds_cosipy[['SWout_albedo', 'SWin_albedo']].resample('1d').sum()
albedo_daily['Albedo'] = albedo_daily['SWout_albedo'] / albedo_daily['SWin_albedo']

# central hours
# select albedo from central hours of the day
albedo_midday = ds_cosipy[['SWout_albedo', 'SWin_albedo']][((ds_cosipy.index.hour > 8) & (ds_cosipy.index.hour <= 13))]
albedo_midday = albedo_midday.resample('1d').sum()
albedo_midday['Albedo'] = albedo_midday['SWout_albedo'] / albedo_midday['SWin_albedo']

# resample back to 10min so it can be merged with other dataframe
albedo_midday_res = albedo_midday['Albedo'].resample('10min').ffill()

# select only full days
ds_cosipy = ds_cosipy[albedo_midday_res.to_frame().index[0]:albedo_midday_res.to_frame().index[-1]]
# add albedo column
ds_cosipy['Albedo'] = albedo_midday_res

C:\Users\Anna\AppData\Local\Temp\ipykernel_25188\665369044.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ds_cosipy['Albedo'] = albedo_midday_res


## Precipitation

In [ ]:
# Add total precipitation from lower station - data not available at WSS
# Data requested from LWD station Weißsee

# Precipitation
filepath = "your own filepath/20170101-20230503_Weißsee_10min_precipitation.dat"
ds_niederschlag = pd.read_csv(filepath)
ds_niederschlag.set_index('datetime', inplace=True)
ds_niederschlag.index = pd.to_datetime(ds_niederschlag.index, format='%Y-%m-%d %H:%M:%S')
ds_niederschlag = ds_niederschlag.rename(columns={"precip": "LWD_obs[mm]"})

# Temperature
filepath = "your own filepath/1_107_Weißsee_Fagge_Lufttemperatur_LT_Basisganglinie.csv"
ds_lufttemperatur = pd.read_csv(filepath, sep = ';', skiprows=11, header=[0], lineterminator='\n', decimal=',', encoding="ISO-8859-1")

cols_to_change = ['Wert']
for col in cols_to_change:
    ds_lufttemperatur[col] = ds_lufttemperatur[col].astype('str')
    ds_lufttemperatur[col] = ds_lufttemperatur[col].str.replace(',', '.')
ds_lufttemperatur = ds_lufttemperatur.replace('---', np.nan)
ds_lufttemperatur['Wert'] = ds_lufttemperatur['Wert'].astype(float)

# set date as index and fix the format. Time is in UTC (checked)
ds_lufttemperatur['datetime'] = ds_lufttemperatur['Datum'] + ' ' + ds_lufttemperatur['Uhrzeit']
ds_lufttemperatur.set_index(ds_lufttemperatur.datetime, inplace=True)
ds_lufttemperatur.index = pd.to_datetime(ds_lufttemperatur.index, format='%d.%m.%Y %H:%M:%S')
ds_lufttemperatur = ds_lufttemperatur.drop(['Datum', 'Uhrzeit', 'datetime'], axis=1)

ds_lufttemperatur = ds_lufttemperatur.rename(columns={"Wert": "Temperature[°C]"}) 
# select only values that have good quality
ds_lufttemperatur = ds_lufttemperatur.where(ds_lufttemperatur["Qualität\r"] == 'Gut\r').dropna()

# Wind speed
#folder pattern where all the datasets are:
filepath = "your own filepath/1_107_Weißsee_Fagge_Windgeschwindigkeit_WG_Basisganglinie.csv"
ds_windspeed = pd.read_csv(filepath, sep = ';', skiprows=11, header=[0], lineterminator='\n', decimal=',', encoding="ISO-8859-1")

cols_to_change = ['Wert[m/s]']
for col in cols_to_change:
    ds_windspeed[col] = ds_windspeed[col].astype('str')
    ds_windspeed[col] = ds_windspeed[col].str.replace(',', '.')
ds_windspeed = ds_windspeed.replace('---', np.nan)
ds_windspeed['Wert[m/s]'] = ds_windspeed['Wert[m/s]'].astype(float)

# set date as index and fix the format. Time is in UTC (checked)
ds_windspeed['datetime'] = ds_windspeed['Datum'] + ' ' + ds_windspeed['Uhrzeit']
ds_windspeed.set_index(ds_windspeed.datetime, inplace=True)
ds_windspeed.index = pd.to_datetime(ds_windspeed.index, format='%d.%m.%Y %H:%M:%S')
ds_windspeed = ds_windspeed.drop(['Datum', 'Uhrzeit', 'datetime'], axis=1)

ds_windspeed = ds_windspeed.rename(columns={"Wert[m/s]": "Wind_speed[m/s]"}) 
# select only values that have good quality
ds_windspeed = ds_windspeed.where(ds_windspeed["Qualität\r"] == 'Gut\r').dropna()

# Cut to the same date range
ds_lufttemperatur = ds_lufttemperatur[datetime.datetime(start_date):datetime.datetime(end_date)]
ds_windspeed = ds_windspeed[datetime.datetime(start_date):datetime.datetime(end_date)]
# And resample to 10 min
ds_lufttemperatur_res = ds_lufttemperatur.resample('10min').ffill()
ds_windspeed_res = ds_windspeed.resample('10min').ffill()

# merge datasets
ds_temp_wind = pd.concat([ds_lufttemperatur_res, ds_windspeed_res], axis = 1)
ds_temp_wind = ds_temp_wind[['Temperature[°C]', 'Wind_speed[m/s]']]
precip_kochendorfer = pd.concat([ds_temp_wind, ds_niederschlag], axis = 1)

In [69]:
# APPLY KOCHENDORFER 2017 CORRECTION - UNDERCATCH
# from kochendorfer 2017, unschielded sensor:
a = 0.063
b = 1.22
c = 0.66
# compute catch efficiency
ce = np.exp( (-1*a*precip_kochendorfer['Wind_speed[m/s]']) * (1 - np.arctan(b*precip_kochendorfer['Temperature[°C]']) + c) )
precip_kochendorfer['precip_koch2017'] = precip_kochendorfer['LWD_obs[mm]'] / ce

# APPLY VERTICAL PRECIPITAITON GRADIENT
# From Winter 2023: HEF range over the year months 8-13% -> Average = 10.5%
gamma_star_hef = 10.5/100
delta_stations = 3499-2480
x_hef = ((gamma_star_hef*delta_stations)/10000) + 1
ds_cosipy.loc['2017-11-01 00:00:00':'2023-02-06 23:50:00','precip_HEFgradient'] = precip_kochendorfer.values * x_hef

C:\Users\Anna\AppData\Local\Temp\ipykernel_25188\3261250760.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ds_cosipy.loc['2017-11-01 00:00:00':'2023-02-06 23:50:00','precip_HEFgradient'] = precip_kochendorfer.values * x_hef


## Snowfall

In [62]:
# Compute snowfall from in situ snowheight data

# More manual corrections: set to nan an isolated observation that irrealistically impacted on the ffill in January and November
ds_cosipy.loc[datetime.datetime(2018,1,4,0,0):datetime.datetime(2018,1,4,0,0), 'snow_clean'] = np.nan
ds_cosipy.loc[datetime.datetime(2018,1,4,12,0):datetime.datetime(2018,1,4,12,0), 'snow_clean'] = np.nan
ds_cosipy.loc[datetime.datetime(2018,1,22,23,0):datetime.datetime(2018,1,22,23,0), 'snow_clean'] = np.nan
ds_cosipy.loc[datetime.datetime(2018,11,18,11,10):datetime.datetime(2018,11,18,11,10), 'snow_clean'] = np.nan
ds_cosipy.loc[datetime.datetime(2022,6,14,13,40):datetime.datetime(2022,6,14,14,0), 'snow_clean'] = np.nan
ds_cosipy.loc[datetime.datetime(2022,6,18,15,0):datetime.datetime(2022,6,18,15,40), 'snow_clean'] = np.nan
ds_cosipy.loc[datetime.datetime(2022,6,30,18,50):datetime.datetime(2022,6,30,19,10), 'snow_clean'] = np.nan
ds_cosipy.loc[datetime.datetime(2022,7,10,6,0):datetime.datetime(2022,7,10,8,0), 'snow_clean'] = np.nan
ds_cosipy.loc[datetime.datetime(2022,7,10,19,30):datetime.datetime(2022,7,10,19,50), 'snow_clean'] = np.nan
ds_cosipy.loc[datetime.datetime(2022,7,12,0,0):datetime.datetime(2022,7,12,2,0), 'snow_clean'] = np.nan
ds_cosipy.loc[datetime.datetime(2022,7,18,4,30):datetime.datetime(2022,7,18,4,30), 'snow_clean'] = np.nan
ds_cosipy.loc[datetime.datetime(2022,7,19,8,30):datetime.datetime(2022,7,19,10,30), 'snow_clean'] = np.nan
ds_cosipy.loc[datetime.datetime(2022,7,19,17,0):datetime.datetime(2022,7,20,0,0), 'snow_clean'] = np.nan
ds_cosipy.loc[datetime.datetime(2022,7,22,0,0):datetime.datetime(2022,7,25,0,0), 'snow_clean'] = np.nan
ds_cosipy.loc[datetime.datetime(2022,7,25,0,0):datetime.datetime(2022,9,1,0,0), 'snow_clean'] = np.nan

# Extract necessary data
snow_base = ds_cosipy[['snow_clean','Tair','Wspeed']]
# Very noisy snowheight still, roll with 20h window and then resample back
snow_rolling = snow_base.rolling('20h', center = True).mean()
snow_rolling = snow_rolling.resample('1h', label = 'right', closed='right').fillna(method='nearest')

# Compute the snow water equivalent using COSIPY's computation of fresh snow density
temp_sh = snow_rolling.snow_clean.diff()
temp_sh = temp_sh.where(temp_sh > 0, 0)
density_fresh_snow = np.maximum(109.0+6.0*(snow_rolling.Tair)+26.0*np.sqrt(snow_rolling.Wspeed), 50.0)
snow_we = temp_sh*(density_fresh_snow/917.)*1000      # mm we, cosipy uses ice density 917kg/m3, WHY???

# Add the computed snowfall to ds_cosipy
ds_cosipy['snowfall'] = temp_sh

C:\Users\Anna\AppData\Local\Temp\ipykernel_25188\86831278.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ds_cosipy['snowfall'] = temp_sh


In [72]:
# Save file
filepath = 'your own filepath/20171031-20231031_WSS_input.csv'
columns = ['Press_Avg', 'Tair', 'Hum', 'Wspeed', 'Wdir', 'SWout_albedo', 'SWin_albedo', 'LWin_corr', 'LWout_corr',
       'Albedo', 'precip_HEFgradient', 'snowfall', 'snow_clean']
ds_cosipy[columns].to_csv(filepath)